# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv file as DataFrame
weather_data = pd.read_csv('../output_data/cities.csv')
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Reconquista,-29.1500,-59.6500,61.18,65,8,5.50,AR,1650113543
1,Butaritari,3.0707,172.7902,80.56,79,66,12.62,KI,1650113543
2,Acarí,-15.4311,-74.6158,68.83,76,100,4.03,PE,1650113543
3,São Joaquim do Monte,-8.4325,-35.8044,84.34,57,36,7.16,BR,1650113543
4,Kiryat Gat,31.6100,34.7642,80.40,22,6,11.27,IL,1650113544


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Create Humidity Heatmap

# Use Lat and Lng as locations and Humidity as weight
locations = weather_data[['Lat', 'Lng']] 
humidity = weather_data['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer (max_intensity at the maximum humidity level of 100
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=3)
 
# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#* Narrow down the DataFrame to find your ideal weather condition. For example:

# * A max temperature lower than 80 degrees but higher than 70.

# * Wind speed less than 10 mph.

# * Zero cloudiness.

# * Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

ideal_weather_data = weather_data[
    (weather_data['Max Temp']<80) & 
    (weather_data['Max Temp'] >70) &
    (weather_data['Wind Speed'] < 10) &
    (weather_data['Cloudiness'] == 0)]


# Resetting index of new DataFrame
ideal_weather_data.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Moindou,-21.6924,165.6770,73.27,82,0,4.85,NC,1650113555
1,Puerto Escondido,15.8500,-97.0667,77.83,78,0,2.30,MX,1650113558
2,Pombia,45.6530,8.6336,78.31,31,0,8.01,IT,1650113338
3,Cassilândia,-19.1133,-51.7342,71.38,54,0,7.72,BR,1650113547
4,Nador,35.1740,-2.9287,77.07,38,0,5.75,MA,1650113577
5,Bullhead City,35.1478,-114.5683,71.02,28,0,7.00,US,1650113579
6,Jardim,-21.4803,-56.1381,71.76,60,0,4.63,BR,1650113588
7,Vao,-22.6667,167.4833,76.87,78,0,6.67,NC,1650113591
8,Aragarças,-15.8975,-52.2508,79.45,61,0,5.75,BR,1650113593
9,Betim,-19.9678,-44.1983,72.84,55,0,3.44,BR,1650113596


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named `hotel_df`
hotel_df = ideal_weather_data.copy()

# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ''
hotel_df = hotel_df.reset_index(drop=True)

In [42]:
# Set parameters to search for hotels with 5000 meters
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params.update({'location': f'{row["Lat"]}, {row["Lng"]}'})
    response = requests.get(base_url, params=params).json()
    results = response.get('results',[])
    if results:
        hotel_name = results[0]['name'] 
    else:
        hotel_name = 'Hotel not Found'
    hotel_df['Hotel Name'][index] = hotel_name
        
hotel_df.reset_index(drop=True)

/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Moindou,-21.6924,165.6770,73.27,82,0,4.85,NC,1650113555,Equi-lodge de Moindou
1,Puerto Escondido,15.8500,-97.0667,77.83,78,0,2.30,MX,1650113558,Aldea del Bazar Hotel and Spa
2,Pombia,45.6530,8.6336,78.31,31,0,8.01,IT,1650113338,Albergo Fontana
3,Cassilândia,-19.1133,-51.7342,71.38,54,0,7.72,BR,1650113547,Delirius Motel
4,Nador,35.1740,-2.9287,77.07,38,0,5.75,MA,1650113577,Hôtel La Giralda
5,Bullhead City,35.1478,-114.5683,71.02,28,0,7.00,US,1650113579,Harrah's Laughlin
6,Jardim,-21.4803,-56.1381,71.76,60,0,4.63,BR,1650113588,Estancia Hotel - Jardim MS
7,Vao,-22.6667,167.4833,76.87,78,0,6.67,NC,1650113591,Hotel Kou-Bugny
8,Aragarças,-15.8975,-52.2508,79.45,61,0,5.75,BR,1650113593,Pousada Tropical
9,Betim,-19.9678,-44.1983,72.84,55,0,3.44,BR,1650113596,Hotel Abba - Betim


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
# Finding out if the hotels and their cities are in the Northern or Southern Hemisphere

northern_hotels = hotel_df.loc[hotel_df['Lat'] >= 0]
northern_hotels

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Puerto Escondido,15.8500,-97.0667,77.83,78,0,2.30,MX,1650113558,Aldea del Bazar Hotel and Spa
2,Pombia,45.6530,8.6336,78.31,31,0,8.01,IT,1650113338,Albergo Fontana
4,Nador,35.1740,-2.9287,77.07,38,0,5.75,MA,1650113577,Hôtel La Giralda
5,Bullhead City,35.1478,-114.5683,71.02,28,0,7.00,US,1650113579,Harrah's Laughlin
10,St. Marys,30.7305,-81.5465,71.62,95,0,4.61,US,1650113609,Spencer House Inn
11,Talavera de la Reina,39.9635,-4.8308,76.84,35,0,3.31,ES,1650113375,Be Live City Center Talavera


In [45]:
# Finding out if the hotels and their cities are in the Northern or Southern Hemisphere

southern_hotels = hotel_df.loc[hotel_df['Lat'] < 0]
southern_hotels

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Moindou,-21.6924,165.6770,73.27,82,0,4.85,NC,1650113555,Equi-lodge de Moindou
3,Cassilândia,-19.1133,-51.7342,71.38,54,0,7.72,BR,1650113547,Delirius Motel
6,Jardim,-21.4803,-56.1381,71.76,60,0,4.63,BR,1650113588,Estancia Hotel - Jardim MS
7,Vao,-22.6667,167.4833,76.87,78,0,6.67,NC,1650113591,Hotel Kou-Bugny
8,Aragarças,-15.8975,-52.2508,79.45,61,0,5.75,BR,1650113593,Pousada Tropical
9,Betim,-19.9678,-44.1983,72.84,55,0,3.44,BR,1650113596,Hotel Abba - Betim


In [50]:
# Setting the ideal humidity level at 50% 

ideal_humidity = weather_data.loc[weather_data['Humidity']<50]
ideal_humidity.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kiryat Gat,31.6100,34.7642,80.40,22,6,11.27,IL,1650113544
1,Alamosa,37.4695,-105.8700,39.06,35,60,7.67,US,1650113544
2,Abnūb,27.2696,31.1511,95.22,7,0,6.38,EG,1650113544
3,Newton,42.3370,-71.2092,60.39,37,0,14.97,US,1650113547
4,Ponta do Sol,32.6667,-17.1000,73.13,34,100,7.16,PT,1650113548
...,...,...,...,...,...,...,...,...,...
103,Mildura,-34.2000,142.1500,64.36,48,5,0.00,AU,1650113615
104,Safaga,26.7292,33.9365,82.35,40,0,10.00,EG,1650113616
105,Baruun-Urt,46.6806,113.2792,46.22,31,100,10.07,MN,1650113616
106,Bagaha,27.1000,84.0833,95.14,7,0,9.51,IN,1650113617


# VacationPy Analysis

- The dataframes and maps of cities around the world which was generated using uniform sampling show that the hotels and ideal weather cities are evenly divided into the Northern (6) and Southern (6) hemispheres.

- Brazil has the most (3) cities in the ideal weather dataframe.

- Apart from 3 cities: Pombia in Italy, Nador in Morocco and Talver de la Reina in Spain all have humidity levels above 50%. Looking at this and the humidity  heatmap there seems to be high levels of humidty around the world. If we set ideal humidity levels at 50%, the original dataframe of 569 is reduced to 108. 